In [59]:
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup

# Importing the data

In [60]:
train = []
with open('test_data.jsonl', 'r') as file:
#     print("Length of dataset", len(list(file)))
    for i in range(20):
        # read a sample of 2 json lines
        train.append(json.loads(file.readline()))

In [61]:
data = pd.DataFrame(columns =['document_text', 'long_answer_candidates', 'question_text', 'annotations', 'document_url', 'example_id'])
for i in range(len(train)):
    data = pd.concat([data,(pd.DataFrame(train[i].items(),index=train[i].keys()).T).iloc[1:]],axis=0)

In [62]:
data = data.reset_index(drop=True)
data = data.drop(columns = ['long_answer_candidates', 'document_url', 'example_id'])
data

,document_text,question_text,annotations
0,Email marketing - Wikipedia <H1> Email marketi...,which is the most common use of opt-in e-mail ...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st..."
1,The Mother ( How I Met Your Mother ) - wikiped...,how i.met your mother who is the mother,"[{'yes_no_answer': 'NONE', 'long_answer': {'st..."
2,Human fertilization - wikipedia <H1> Human fer...,what type of fertilisation takes place in humans,"[{'yes_no_answer': 'NONE', 'long_answer': {'st..."
3,List of National Football League career quarte...,who had the most wins in the nfl,"[{'yes_no_answer': 'NONE', 'long_answer': {'st..."
4,Roanoke Colony - wikipedia <H1> Roanoke Colony...,what happened to the lost settlement of roanoke,"[{'yes_no_answer': 'NONE', 'long_answer': {'st..."
5,List of regions of Africa - wikipedia <H1> Lis...,what are the different regions of africa and h...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st..."
6,Pom Klementieff - wikipedia <H1> Pom Klementie...,who played mantis guardians of the galaxy 2,"[{'yes_no_answer': 'NONE', 'long_answer': {'st..."
7,Frosty the Snowman ( film ) - wikipedia <H1> F...,who did the voice of the magician in frosty th...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st..."
8,History of the Acadians - wikipedia <H1> Histo...,what indian tribe did the acadians form friend...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st..."
9,Outer Banks - wikipedia <H1> Outer Banks </H1>...,what is considered the outer banks in north ca...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st..."


In [63]:
short=[]
for i in range(len(data)):
    output = data['document_text'][i].split(' ')
    if data['annotations'][i][0]['short_answers']!=[]:
        short.append(' '.join(output[data['annotations'][i][0]['short_answers'][0]['start_token']:data['annotations'][i][0]['short_answers'][0]['end_token']]))
    else:
        short.append('NaN')

In [65]:
data['short_answers'] = short

In [66]:
data = data.drop(columns=['annotations'])

In [67]:
data['document_text'][0]

"Email marketing - Wikipedia <H1> Email marketing </H1> Jump to : navigation , search <Table> <Tr> <Td> </Td> <Td> ( hide ) This article has multiple issues . Please help improve it or discuss these issues on the talk page . ( Learn how and when to remove these template messages ) <Table> <Tr> <Td> </Td> <Td> This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed . ( September 2014 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <Td> </Td> <Td> This article possibly contains original research . Please improve it by verifying the claims made and adding inline citations . Statements consisting only of original research should be removed . ( January 2015 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <T

# Cleaning the Html tags

In [68]:
data.head()

,document_text,question_text,short_answers
0,Email marketing - Wikipedia <H1> Email marketi...,which is the most common use of opt-in e-mail ...,a newsletter sent to an advertising firm 's cu...
1,The Mother ( How I Met Your Mother ) - wikiped...,how i.met your mother who is the mother,Tracy McConnell
2,Human fertilization - wikipedia <H1> Human fer...,what type of fertilisation takes place in humans,NaN
3,List of National Football League career quarte...,who had the most wins in the nfl,Tom Brady
4,Roanoke Colony - wikipedia <H1> Roanoke Colony...,what happened to the lost settlement of roanoke,NaN


## Get title

In [69]:
import re
pattern="(<H1>)(.*)(</H1>)"

title = []
for text in data['document_text']:
    #print(text)
    g=re.search(pattern,text)
    t = g.group(2).strip()
    if t:
        title.append(t)
    else:
        title.append("")
data['title'] = title
data.head()

,document_text,question_text,short_answers,title
0,Email marketing - Wikipedia <H1> Email marketi...,which is the most common use of opt-in e-mail ...,a newsletter sent to an advertising firm 's cu...,Email marketing
1,The Mother ( How I Met Your Mother ) - wikiped...,how i.met your mother who is the mother,Tracy McConnell,The Mother ( How I Met Your Mother )
2,Human fertilization - wikipedia <H1> Human fer...,what type of fertilisation takes place in humans,NaN,Human fertilization
3,List of National Football League career quarte...,who had the most wins in the nfl,Tom Brady,List of National Football League career quarte...
4,Roanoke Colony - wikipedia <H1> Roanoke Colony...,what happened to the lost settlement of roanoke,NaN,Roanoke Colony


In [91]:
import requests
from bs4 import BeautifulSoup

para = []
for txt in data['document_text']:
    each_text = txt.split('<P>')
    text = [BeautifulSoup(i, 'lxml').get_text() for i in each_text]
    para.append(text)
data['paragraphs'] = para
data = data.drop(columns = ['document_text'])

In [92]:
data.head()

,question_text,short_answers,title,paragraphs
0,which is the most common use of opt-in e-mail ...,a newsletter sent to an advertising firm 's cu...,Email marketing,[Email marketing - Wikipedia Email marketing ...
1,how i.met your mother who is the mother,Tracy McConnell,The Mother ( How I Met Your Mother ),[The Mother ( How I Met Your Mother ) - wikipe...
2,what type of fertilisation takes place in humans,NaN,Human fertilization,[Human fertilization - wikipedia Human fertil...
3,who had the most wins in the nfl,Tom Brady,List of National Football League career quarte...,[List of National Football League career quart...
4,what happened to the lost settlement of roanoke,NaN,Roanoke Colony,[Roanoke Colony - wikipedia Roanoke Colony J...


In [94]:
data = data[~data.short_answers.str.contains("NaN")]
data

,question_text,short_answers,title,paragraphs
0,which is the most common use of opt-in e-mail ...,a newsletter sent to an advertising firm 's cu...,Email marketing,[Email marketing - Wikipedia Email marketing ...
1,how i.met your mother who is the mother,Tracy McConnell,The Mother ( How I Met Your Mother ),[The Mother ( How I Met Your Mother ) - wikipe...
3,who had the most wins in the nfl,Tom Brady,List of National Football League career quarte...,[List of National Football League career quart...
6,who played mantis guardians of the galaxy 2,Pom Klementieff,Pom Klementieff,[Pom Klementieff - wikipedia Pom Klementieff ...
15,the nashville sound brought a polished and cos...,the use of lush string arrangements with a rea...,Nashville sound,[Nashville sound - wikipedia Nashville sound ...
19,what channel is the premier league on in france,SFR Sport,List of Premier League broadcasters,[List of Premier League broadcasters - wikiped...


In [95]:
data.to_csv("dataset.csv", index=False)